# Exercise 4.8

## Step 1:

Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [4]:
import pandas as pd
import numpy as np
import os

In [5]:
path = r'C:\Users\Jenna\Desktop\Data Course\Part4_Instacart_analysis\02_Data'

In [6]:
df = pd.read_pickle(os.path.join(path, 'Prepared_Data', 'orders_products_merged_update.pkl'))

## Step 2:

In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [7]:
df.groupby('department_id').agg({'user_order_number':['mean']})

user_order_number
                           mean
department_id                  
1                     15.457838
2                     17.277920
3                     17.170395
4                     17.811403
5                     15.215751
6                     16.439806
7                     17.225802
8                     15.340650
9                     15.895474
10                    20.197148
11                    16.170638
12                    15.887671
13                    16.583536
14                    16.773669
15                    16.165037
16                    17.665606
17                    15.694469
18                    19.310397
19                    17.177343
20                    16.473447
21                    22.902379

## Step 3: Analyze the result. How do the results for the entire dataframe differ from those of the subset? 


The means are slighly different and more department_ids are visible with the entire database but overall the out put is similar

## Step 4: Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [9]:
# first generate a max order column that shows the maximum order per customer

df['max_order'] = df.groupby(['user_id'])['user_order_number'].transform(np.max)

In [10]:
# second is to check the head to see if the column is correct

df.head(15)

order_id  user_id  user_order_number  orders_day_of_week  \
0    2539329        1                  1                   2   
1    2398795        1                  2                   3   
2     473747        1                  3                   3   
3    2254736        1                  4                   4   
4     431534        1                  5                   4   
5    3367565        1                  6                   2   
6     550135        1                  7                   1   
7    3108588        1                  8                   1   
8    2295261        1                  9                   1   
9    2550362        1                 10                   4   
10   2968173       15                 15                   1   
11   1870022       15                 17                   2   
12   1911383       15                 18                   2   
13   2715276       15                 21                   1   
14    487368       15                 22                   1   

    order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                   8                     NaN         196                  1   
1                   7                    15.0         196                  1   
2                  12                    21.0         196                  1   
3                   7                    29.0         196                  1   
4                  15                    28.0         196                  1   
5                   7                    19.0         196                  1   
6                   9                    20.0         196                  1   
7                  14                    14.0         196                  2   
8                  16                     0.0         196                  4   
9                   8                    30.0         196                  1   
10                  9                     7.0         196                  2   
11                 16                     8.0         196                  6   
12                 11                     7.0         196                  1   
13                  9                     7.0         196                  1   
14                 10                    14.0         196                  1   

    reordered product_name  aisle_id  department_id  prices  \
0           0         Soda        77              7     9.0   
1           1         Soda        77              7     9.0   
2           1         Soda        77              7     9.0   
3           1         Soda        77              7     9.0   
4           1         Soda        77              7     9.0   
5           1         Soda        77              7     9.0   
6           1         Soda        77              7     9.0   
7           1         Soda        77              7     9.0   
8           1         Soda        77              7     9.0   
9           1         Soda        77              7     9.0   
10          0         Soda        77              7     9.0   
11          1         Soda        77              7     9.0   
12          1         Soda        77              7     9.0   
13          1         Soda        77              7     9.0   
14          1         Soda        77              7     9.0   

      price_range_loc     busiest_day    busiest_days busiest_period_of_day  \
0   Mid range product  Regularly Busy  Regularly Busy        Average Orders   
1   Mid range product  Regularly Busy    Slowest days        Average Orders   
2   Mid range product  Regularly Busy    Slowest days           Most Orders   
3   Mid range product      Least Busy    Slowest days        Average Orders   
4   Mid range product      Least Busy    Slowest days           Most Orders   
5   Mid range product  Regularly Busy  Regularly Busy        Average Orders   
6   Mid range product  Regularly Busy    Busiest days           Most Orders   
7   Mid range product  Regularly Busy    Busiest days    

### after the max order column is created, the conditions for loyalty flags can be written

In [12]:
# first condition, customers with high orders

df.loc[df['max_order'] >40 , 'loyalty_flag'] = 'Loyal Customer'

In [16]:
# second condition, customers with regular order numbers

df.loc[(df['max_order']<= 40) & (df['max_order']>10), 'loyalty_flag'] = 'Regular Customer'

In [17]:
# thrid condition, new customers

df.loc[df['max_order']<=10, 'loyalty_flag']= 'New Customer'

In [18]:
# check the value counts for the loyalty flag column

df['loyalty_flag'].value_counts(dropna = False)

Regular Customer    15876776
Loyal Customer      10284093
New Customer         6243990
Name: loyalty_flag, dtype: int64

## Step 5:

The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [19]:
# in this case we will group by loyalty flag and use the aggregate function to find the stats

df.groupby('loyalty_flag').agg({'prices': ['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal Customer    10.386336  1.0  99999.0
New Customer      13.294670  1.0  99999.0
Regular Customer  12.495717  1.0  99999.0

## Step 6:

The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [20]:
# first create a column that shows the average price of orders per customer

df['average_order'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [22]:
# Check the output

df.head(15)

order_id  user_id  user_order_number  orders_day_of_week  \
0    2539329        1                  1                   2   
1    2398795        1                  2                   3   
2     473747        1                  3                   3   
3    2254736        1                  4                   4   
4     431534        1                  5                   4   
5    3367565        1                  6                   2   
6     550135        1                  7                   1   
7    3108588        1                  8                   1   
8    2295261        1                  9                   1   
9    2550362        1                 10                   4   
10   2968173       15                 15                   1   
11   1870022       15                 17                   2   
12   1911383       15                 18                   2   
13   2715276       15                 21                   1   
14    487368       15                 22                   1   

    order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                   8                     NaN         196                  1   
1                   7                    15.0         196                  1   
2                  12                    21.0         196                  1   
3                   7                    29.0         196                  1   
4                  15                    28.0         196                  1   
5                   7                    19.0         196                  1   
6                   9                    20.0         196                  1   
7                  14                    14.0         196                  2   
8                  16                     0.0         196                  4   
9                   8                    30.0         196                  1   
10                  9                     7.0         196                  2   
11                 16                     8.0         196                  6   
12                 11                     7.0         196                  1   
13                  9                     7.0         196                  1   
14                 10                    14.0         196                  1   

    reordered product_name  aisle_id  department_id  prices  \
0           0         Soda        77              7     9.0   
1           1         Soda        77              7     9.0   
2           1         Soda        77              7     9.0   
3           1         Soda        77              7     9.0   
4           1         Soda        77              7     9.0   
5           1         Soda        77              7     9.0   
6           1         Soda        77              7     9.0   
7           1         Soda        77              7     9.0   
8           1         Soda        77              7     9.0   
9           1         Soda        77              7     9.0   
10          0         Soda        77              7     9.0   
11          1         Soda        77              7     9.0   
12          1         Soda        77              7     9.0   
13          1         Soda        77              7     9.0   
14          1         Soda        77              7     9.0   

      price_range_loc     busiest_day    busiest_days busiest_period_of_day  \
0   Mid range product  Regularly Busy  Regularly Busy        Average Orders   
1   Mid range product  Regularly Busy    Slowest days        Average Orders   
2   Mid range product  Regularly Busy    Slowest days           Most Orders   
3   Mid range product      Least Busy    Slowest days        Average Orders   
4   Mid range product      Least Busy    Slowest days           Most Orders   
5   Mid range product  Regularly Busy  Regularly Busy        Average Orders   
6   Mid range product  Regularly Busy    Busiest days           Most Orders   
7   Mid range product  Regularly Busy    Busiest days    

### After average orders are calculated, flags can be made

In [23]:
# first condition for low order prices

df.loc[df['average_order']<10, 'spending_flag'] = 'Low Spender'

In [25]:
# second condition for high average order prices

df.loc[df['average_order'] >= 10, 'spending_flag'] = 'High Spender'

In [26]:
# check the output

df['spending_flag'].value_counts(dropna = False)

Low Spender     31770614
High Spender      634245
Name: spending_flag, dtype: int64

## Step 7:

In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”


In [29]:
# similarly to above we first will make a column that displays the median day since prior order

df['order_freq']= df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [30]:
# check the output

df.head(15)

order_id  user_id  user_order_number  orders_day_of_week  \
0    2539329        1                  1                   2   
1    2398795        1                  2                   3   
2     473747        1                  3                   3   
3    2254736        1                  4                   4   
4     431534        1                  5                   4   
5    3367565        1                  6                   2   
6     550135        1                  7                   1   
7    3108588        1                  8                   1   
8    2295261        1                  9                   1   
9    2550362        1                 10                   4   
10   2968173       15                 15                   1   
11   1870022       15                 17                   2   
12   1911383       15                 18                   2   
13   2715276       15                 21                   1   
14    487368       15                 22                   1   

    order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                   8                     NaN         196                  1   
1                   7                    15.0         196                  1   
2                  12                    21.0         196                  1   
3                   7                    29.0         196                  1   
4                  15                    28.0         196                  1   
5                   7                    19.0         196                  1   
6                   9                    20.0         196                  1   
7                  14                    14.0         196                  2   
8                  16                     0.0         196                  4   
9                   8                    30.0         196                  1   
10                  9                     7.0         196                  2   
11                 16                     8.0         196                  6   
12                 11                     7.0         196                  1   
13                  9                     7.0         196                  1   
14                 10                    14.0         196                  1   

    reordered product_name  ...  prices    price_range_loc     busiest_day  \
0           0         Soda  ...     9.0  Mid range product  Regularly Busy   
1           1         Soda  ...     9.0  Mid range product  Regularly Busy   
2           1         Soda  ...     9.0  Mid range product  Regularly Busy   
3           1         Soda  ...     9.0  Mid range product      Least Busy   
4           1         Soda  ...     9.0  Mid range product      Least Busy   
5           1         Soda  ...     9.0  Mid range product  Regularly Busy   
6           1         Soda  ...     9.0  Mid range product  Regularly Busy   
7           1         Soda  ...     9.0  Mid range product  Regularly Busy   
8           1         Soda  ...     9.0  Mid range product  Regularly Busy   
9           1         Soda  ...     9.0  Mid range product      Least Busy   
10          0         Soda  ...     9.0  Mid range product  Regularly Busy   
11          1         Soda  ...     9.0  Mid range product  Regularly Busy   
12          1         Soda  ...     9.0  Mid range product  Regularly Busy   
13          1         Soda  ...     9.0  Mid range product  Regularly Busy   
14          1         Soda  ...     9.0  Mid range product  Regularly Busy   

      busiest_days busiest_period_of_day max_order      loyalty_flag  \
0   Regularly Busy        Average Orders        10      New Customer   
1     Slowest days        Average Orders        10      New Customer   
2     Slowest days           Most Orders        10      New Customer   
3     Slowest days        Average Orders        10      New Customer   
4     Slowest days           Most Orders        10      New Customer   
5   Regularly Bus

### Once the median is available, the flag can be made

In [36]:
# condition one, non freq customer

df.loc[df['order_freq'] >20, 'freq_flag'] = 'Non-frequent Customer'

In [32]:
# condition 2, regular customer

df.loc[(df['order_freq'] > 10) & (df['order_freq'] <= 20), 'freq_flag'] = 'Regular Customer'

In [33]:
# condition 3, frequent customer

df.loc[df['order_freq'] <= 10, 'freq_flag'] = 'Frequent Customer'

In [37]:
# check output

df['freq_flag'].value_counts(dropna = False)

Frequent Customer        21559853
Regular Customer          7208564
Non-frequent Customer     3636437
NaN                             5
Name: freq_flag, dtype: int64

## checking the null rows 

In [38]:
nan_rows = df[df['freq_flag'].isnull()]
nan_rows

order_id  user_id  user_order_number  orders_day_of_week  \
13645692    895835   159838                  1                   0   
17251990    895835   159838                  1                   0   
17622767    895835   159838                  1                   0   
24138593    895835   159838                  1                   0   
25880002    895835   159838                  1                   0   

          order_hour_of_day  days_since_prior_order  product_id  \
13645692                 17                     NaN       10749   
17251990                 17                     NaN       33401   
17622767                 17                     NaN       23695   
24138593                 17                     NaN       21334   
25880002                 17                     NaN       22198   

          add_to_cart_order  reordered  \
13645692                  3          0   
17251990                  6          0   
17622767                  2          0   
24138593                  5          0   
25880002                  1          0   

                                               product_name  ...  \
13645692                            Organic Red Bell Pepper  ...   
17251990                               Goat Cheese Crumbles  ...   
17622767                           California Veggie Burger  ...   
24138593                              Organic Peeled Garlic  ...   
25880002  4X Ultra Concentrated Natural Laundry Detergen...  ...   

            price_range_loc  busiest_day  busiest_days busiest_period_of_day  \
13645692  Mid range product  Busiest Day  Busiest days        Average Orders   
17251990  Mid range product  Busiest Day  Busiest days        Average Orders   
17622767  Low range product  Busiest Day  Busiest days        Average Orders   
24138593  Mid range product  Busiest Day  Busiest days        Average Orders   
25880002  Low range product  Busiest Day  Busiest days        Average Orders   

         max_order  loyalty_flag average_order  spending_flag order_freq  \
13645692         1  New Customer          7.42    Low Spender        NaN   
17251990         1  New Customer          7.42    Low Spender        NaN   
17622767         1  New Customer          7.42    Low Spender        NaN   
24138593         1  New Customer          7.42    Low Spender        NaN   
25880002         1  New Customer          7.42    Low Spender        NaN   

          freq_flag  
13645692        NaN  
17251990        NaN  
17622767        NaN  
24138593        NaN  
25880002        NaN  

[5 rows x 23 columns]

## Null rows appear to be one user id who does not have data for prior orders

## check header of final df

In [39]:
df.head(20)

order_id  user_id  user_order_number  orders_day_of_week  \
0    2539329        1                  1                   2   
1    2398795        1                  2                   3   
2     473747        1                  3                   3   
3    2254736        1                  4                   4   
4     431534        1                  5                   4   
5    3367565        1                  6                   2   
6     550135        1                  7                   1   
7    3108588        1                  8                   1   
8    2295261        1                  9                   1   
9    2550362        1                 10                   4   
10   2968173       15                 15                   1   
11   1870022       15                 17                   2   
12   1911383       15                 18                   2   
13   2715276       15                 21                   1   
14    487368       15                 22                   1   
15   2293453       19                  2                   5   
16   1973799       19                  5                   6   
17    532817       19                  7                   4   
18   1573906       21                 10                   3   
19   1593000       31                 10                   3   

    order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                   8                     NaN         196                  1   
1                   7                    15.0         196                  1   
2                  12                    21.0         196                  1   
3                   7                    29.0         196                  1   
4                  15                    28.0         196                  1   
5                   7                    19.0         196                  1   
6                   9                    20.0         196                  1   
7                  14                    14.0         196                  2   
8                  16                     0.0         196                  4   
9                   8                    30.0         196                  1   
10                  9                     7.0         196                  2   
11                 16                     8.0         196                  6   
12                 11                     7.0         196                  1   
13                  9                     7.0         196                  1   
14                 10                    14.0         196                  1   
15                 14                     6.0         196                  3   
16                 12                     8.0         196                 15   
17                 17                     6.0         196                  1   
18                 10                     6.0         196                  2   
19                  8                     7.0         196                 17   

    reordered product_name  ...    price_range_loc     busiest_day  \
0           0         Soda  ...  Mid range product  Regularly Busy   
1           1         Soda  ...  Mid range product  Regularly Busy   
2           1         Soda  ...  Mid range product  Regularly Busy   
3           1         Soda  ...  Mid range product      Least Busy   
4           1         Soda  ...  Mid range product      Least Busy   
5           1         Soda  ...  Mid range product  Regularly Busy   
6           1         Soda  ...  Mid range product  Regularly Busy   
7           1         Soda  ...  Mid range product  Regularly Busy   
8           1         Soda  ...  Mid range product  Regularly Busy   
9           1         Soda  ...  Mid range product      Least Busy   
10          0         Soda  ...  Mid range product  Regularly Busy   
11          1         Soda  ...  Mid range product  Regularly Busy   
12          1         Soda  ...  Mid range product  Regularly Busy  

## Step 9: Export

In [ ]:
df.to_pickle(os.path.join(path, 'Prepared_Data', 'merged_exercise8_update.pkl'))